# Лабораторна робота 2

**Тема:** Прийняття рішення в умовах невизначеності за класичними критеріями.

**Мета:** дослідження питань ухвалення рішення в умовах, коли вибір
деякої стратегії гарантує одержання бажаного результату з певною ймовірністю
залежно від невизначеного стану середовища.

**Виконав:** студент 321 групи, Ткачук І.В.

## Завдання 

### Варіант 6

У сільському господарстві з посівною площею 1430 га вирішено
побудувати елеватор за одним з типових проектів на 20, 30, 40, 50 або 60
тис. центнерів зерна. Будівництво елеватора обійдеться в 370000 грн.
Вартість матеріалів і устаткування для елеватора потужністю 20 тис.
дорівнює 600000 грн. і зростає на 10% із зростанням об'єму на 10 тис.
Витрати на експлуатацію елеватора на 20 тис. дорівнюють 200 000 грн. і
зростають на 10000 грн. із зростанням об'єму на 10 тис. За зберігання зерна
на рахунок елеватора вноситься плата 100 грн за центнер. Зібрано
статистику врожайності за останні 10 років. Який із проектів вибрати?

| Рік | 2008 | 2009 | 2010 | 2011 | 2012 | 2013 | 2014 | 2015 | 2016 | 2017 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Урожайність, ц/га | 14.4 | 20.1 | 15.5 | 34.5 | 55.4 | 26.2 | 12.2 | 76.0 | 22.5| 33.5 |
| Середньорічна температура | 5.5 | 5.4 | 5.9 | 5.2 | 4.8 | 5.3 | 6.0 | 4.8 | 4.7 | 4.5 |


In [13]:
import pandas as pd

yield_data = {
    'Рік': [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
    'Урожай': [14.4, 20.1, 15.5, 34.5, 55.4, 26.2, 12.2, 76.0, 25.5, 33.5],
    'Температура': [5.5, 5.4, 5.9, 5.2, 4.8, 5.3, 6.0, 4.8, 4.7, 4.5]
}


df = pd.DataFrame(yield_data)

df

,Рік,Урожай,Температура
0,2008,14.4,5.5
1,2009,20.1,5.4
2,2010,15.5,5.9
3,2011,34.5,5.2
4,2012,55.4,4.8
5,2013,26.2,5.3
6,2014,12.2,6.0
7,2015,76.0,4.8
8,2016,25.5,4.7
9,2017,33.5,4.5


In [14]:
sown_area_ha = 1430

elavator_capacities = [20_000, 30_000, 40_000, 50_000, 60_000]
construction_cost = 370_000
base_material_cost = 600_000 
base_operating_cost = 200_000
fee_per_centner = 100


df['Загальний урожай'] = df['Урожай'] * sown_area_ha
df['Дохід за урожай'] = df['Загальний урожай'] * fee_per_centner

df.head()

,Рік,Урожай,Температура,Загальний урожай,Дохід за урожай
0,2008,14.4,5.5,20592.0,2059200.0
1,2009,20.1,5.4,28743.0,2874300.0
2,2010,15.5,5.9,22165.0,2216500.0
3,2011,34.5,5.2,49335.0,4933500.0
4,2012,55.4,4.8,79222.0,7922200.0


In [15]:
costs = {
    "Потужність": [],
    "Вартість устаткування": [],
    "Вартість експлутації": [],
    "Загальна вартість": [],
    "Дохід": [],
    "Прибуток": [],
}

# коефіцієнт довіри
alpha = 0.5

for capacity in elavator_capacities:
    material_equipment_cost = base_material_cost * (1 + 0.1 * ((capacity / 20_000) - 1))
    operating_cost = base_operating_cost + (capacity - 20_000) / 10_000 * 10_000
    total_cost = construction_cost + material_equipment_cost + operating_cost

    costs["Потужність"].append(capacity)
    costs["Вартість устаткування"].append(material_equipment_cost)
    costs["Вартість експлутації"].append(operating_cost)
    costs["Загальна вартість"].append(total_cost)
    
    revenue = min(df['Загальний урожай'].mean(), capacity) * fee_per_centner
    total_cost = costs['Загальна вартість'][elavator_capacities.index(capacity)]
    profit = revenue - total_cost
    
    costs['Дохід'].append(revenue)
    costs['Прибуток'].append(profit)

costs_df = pd.DataFrame(costs)
costs_df

,Потужність,Вартість устаткування,Вартість експлутації,Загальна вартість,Дохід,Прибуток
0,20000,600000.0,200000.0,1170000.0,2000000.0,830000.0
1,30000,630000.0,210000.0,1210000.0,3000000.0,1790000.0
2,40000,660000.0,220000.0,1250000.0,4000000.0,2750000.0
3,50000,690000.0,230000.0,1290000.0,4480190.0,3190190.0
4,60000,720000.0,240000.0,1330000.0,4480190.0,3150190.0


Знайдемо критерії Вальда, Гурвіца, Лапласа та Севіджа:

In [16]:
alpha = 0.5

criteries = {'Вальд': [], 'Гурвіц': [], 'Лаплас': [], 'Севідж': []}

criteries['Вальд'] = [min(costs_df['Прибуток'])]

for capacity in elavator_capacities:
    min_profit = min(costs_df["Прибуток"])
    max_profit = max(costs_df["Прибуток"])

    # критерій гурвіца
    hurwitz_value = alpha * max_profit + (1 - alpha) * min_profit
    criteries['Гурвіц'].append(hurwitz_value)

    laplace_value = costs_df["Прибуток"].mean()
    criteries['Лаплас'].append(laplace_value)
    
# Матриця жалів
regret_matrix = pd.DataFrame()
for capacity in elavator_capacities:
    regret_matrix[capacity] = [max(costs_df['Прибуток']) - profit for profit in costs_df['Прибуток']]

criteries['Севідж'] = regret_matrix.max().tolist()
    
criteries


{'Вальд': [830000.0],
 'Гурвіц': [2010095.0, 2010095.0, 2010095.0, 2010095.0, 2010095.0],
 'Лаплас': [2342076.0, 2342076.0, 2342076.0, 2342076.0, 2342076.0],
 'Севідж': [2360190.0, 2360190.0, 2360190.0, 2360190.0, 2360190.0]}

In [17]:
temperature_ranges = {
    'Мінімальна температура': df['Температура'].min(),
    'Середня температура': df['Температура'].mean(),
    'Максимальна температура': df['Температура'].max()
}

temperature_ranges

{'Мінімальна температура': 4.5,
 'Середня температура': 5.21,
 'Максимальна температура': 6.0}

In [18]:
yield_categories = ['Min Урожай', 'Medium Урожай', 'High Урожай']
yield_threshold = [df['Урожай'].min(), df['Урожай'].mean(), df['Урожай'].max()]

frequency_table = {
    state: {category: 0 for category in yield_categories} for state in temperature_ranges
}

for index, row in df.iterrows():
    temperature = row['Температура']
    yield_amount = row['Урожай']

    # Determine temperature range
    for range_name, temp in temperature_ranges.items():
        if temperature <= temp:
            temp_range = range_name
            break
        
    if yield_amount < yield_threshold[0]:
        yield_category = 'Min Урожай'
    elif yield_amount < yield_threshold[1]:
        yield_category = 'Medium Урожай'
    else:
        yield_category = 'High Урожай'
    
    frequency_table[temp_range][yield_category] += 1

    
frequency = pd.DataFrame(frequency_table)

probabilities = frequency.div(frequency.sum(axis=0), axis=1)

probabilities



,Мінімальна температура,Середня температура,Максимальна температура
Min Урожай,0.0,0.00,0.0
Medium Урожай,0.0,0.25,1.0
High Урожай,1.0,0.75,0.0


## Контрольні питання

1. Сформулюйте формально модель задачі прийняття рішення в
невизначеності.
2. Що називається функцією корисності?
3. Яким аксіомам задовольняє функція корисності $U(x_i)$.
4. Як визначається ймовірність досягнення результату $O_j$ в залежності від
стану середовища?
5. Сформулювати класичні критерії прийняття рішення: Вальда, Гурвіца,
Севіджа, Лапласа.

### 1. **Формальна модель задачі прийняття рішення в невизначеності:**

- **Елементи моделі:** Маємо набір стратегій або дій та набір можливих станів світу (або результатів).
- **Невизначеність:** Відсутні точні ймовірності для настання кожного зі станів світу.
- **Рішення**: Вибір стратегії відбувається з використанням різних критеріїв оцінки ризику та невизначеності, наприклад, мінімаксного (Вальда) або мінімаксного жалю (Севіджа).


### 2. **Функція корисності**:

Це математична функція, яка відображає величину задоволення або вигоди, отриманої від різних результатів або наборів результатів. Функція корисності дозволяє кількісно оцінити переваги та зробити обґрунтований вибір.

### 3. **Аксіоми функції корисності $U(x_i)$**:

- **Повнота**: Кожна пара результатів може бути порівняна.
- **Транзитивність**: Якщо результат $A$ переважає над $B$, і $B$ переважає над $C$, то $A$ переважає над $C$.
- **Неперервність**: Невеликі зміни в результаті призводять до невеликих змін у корисності.
- **Незалежність**: Переваги між результатами є незалежними від інших факторів.

4. **Визначення ймовірності досягнення результату $O_j$ в залежності від стану середовища**:

В умовах невизначеності точні ймовірності для результатів зазвичай невідомі. Замість цього, можуть використовуватися суб'єктивні оцінки ймовірностей або різні евристичні підходи для визначення найбільш імовірних результатів.

5. **Класичні критерії прийняття рішення**:

- **Критерій Вальда (мінімакс)**: Вибрати стратегію, яка максимізує мінімальний можливий виграш.
- **Критерій Гурвіца**: Вибрати стратегію на основі взваженого середнього максимального та мінімального виграшу для кожної стратегії.
- **Критерій Севіджа (мінімаксний жаль)**: Вибрати стратегію, яка мінімізує максимальний можливий жаль (різниця між виграшем, що міг бути отриманий, і виграшем, який було отримано).
- **Критерій Лапласа**: Вибрати стратегію з найвищою середньою виплатою, розрахованою як середнє арифметичне всіх виплат для стратегії.